<a href="https://colab.research.google.com/github/Saranyanv/test/blob/master/test_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Preparation**




In [1]:
 import tensorflow as tf
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size 
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [0]:
from IPython.display import clear_output, Image, display, HTML
import datetime

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def
  
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [0]:
import numpy as np
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [0]:
n_inputs = 28*28  # MNIST
n_hidden1_1 = 300
n_hidden1_2 = 100
n_hidden2_1 = 200
n_hidden2_2 = 100
n_outputs = 10

In [0]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

**Model 1 | **
**Hidden layer 1 - 300 nodes / Sigmoid Activation | ** 
**Hidden layer 2 - 100 nodes / Sigmoid Activation**


In [0]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [0]:
with tf.name_scope("dnn1"):
    hidden1 = tf.layers.dense(X, n_hidden1_1, name="hidden1",
                              activation=tf.nn.sigmoid, use_bias=True)
    hidden2 = tf.layers.dense(hidden1, n_hidden1_2, name="hidden2",
                              activation=tf.nn.sigmoid, use_bias=True)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

In [0]:
with tf.name_scope("loss1"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [0]:
learning_rate = 0.01

with tf.name_scope("train1"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval1"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [13]:
n_epochs = 120
batch_size = 500
final_model_path = "./model1"

start_tm=datetime.datetime.now()
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        saver.save(sess, final_model_path)
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)
end_tm=datetime.datetime.now()
print ("Time taken to fit is ",end_tm - start_tm)

0 Batch accuracy: 0.134 Validation accuracy: 0.123
1 Batch accuracy: 0.178 Validation accuracy: 0.1314
2 Batch accuracy: 0.184 Validation accuracy: 0.1822
3 Batch accuracy: 0.27 Validation accuracy: 0.2818
4 Batch accuracy: 0.394 Validation accuracy: 0.4044
5 Batch accuracy: 0.346 Validation accuracy: 0.386
6 Batch accuracy: 0.414 Validation accuracy: 0.401
7 Batch accuracy: 0.466 Validation accuracy: 0.4746
8 Batch accuracy: 0.524 Validation accuracy: 0.4956
9 Batch accuracy: 0.52 Validation accuracy: 0.5646
10 Batch accuracy: 0.524 Validation accuracy: 0.5664
11 Batch accuracy: 0.534 Validation accuracy: 0.5704
12 Batch accuracy: 0.56 Validation accuracy: 0.5842
13 Batch accuracy: 0.6 Validation accuracy: 0.588
14 Batch accuracy: 0.598 Validation accuracy: 0.606
15 Batch accuracy: 0.608 Validation accuracy: 0.5962
16 Batch accuracy: 0.602 Validation accuracy: 0.6218
17 Batch accuracy: 0.592 Validation accuracy: 0.616
18 Batch accuracy: 0.58 Validation accuracy: 0.6304
19 Batch accura

KeyboardInterrupt: ignored

In [0]:
with tf.Session() as sess:
    saver.restore(sess, final_model_path)
    train_Accuracy_val = accuracy.eval(feed_dict={X: X_train, y: y_train})
    test_Accuracy_val = accuracy.eval(feed_dict={X: X_test, y: y_test})
    
print ("Training Accuracy=",train_Accuracy_val)
print ("Testing Accuracy=",test_Accuracy_val)

INFO:tensorflow:Restoring parameters from ./model1
Training Accuracy= 0.11234546
Testing Accuracy= 0.1135


In [0]:
#show_graph(tf.get_default_graph())
show_graph(tf.get_default_graph().as_graph_def())

In [0]:
reset_graph()
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [0]:
with tf.name_scope("dnn2"):
    hidden1 = tf.layers.dense(X, n_hidden1_1, name="hidden1",
                              activation=tf.nn.relu, use_bias=True)
    hidden2 = tf.layers.dense(hidden1, n_hidden1_2, name="hidden2",
                              activation=tf.nn.relu, use_bias=True)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

In [0]:
with tf.name_scope("loss2"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")   

In [0]:
learning_rate = 0.01

with tf.name_scope("train2"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval2"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [0]:
n_epochs = 120
batch_size = 500
final_model_path = "./model2"

start_tm=datetime.datetime.now()
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        saver.save(sess, final_model_path)
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)
end_tm=datetime.datetime.now()
print ("Time taken to fit is ",end_tm - start_tm)

0 Batch accuracy: 0.684 Validation accuracy: 0.6906
1 Batch accuracy: 0.812 Validation accuracy: 0.7978
2 Batch accuracy: 0.832 Validation accuracy: 0.839
3 Batch accuracy: 0.876 Validation accuracy: 0.8628
4 Batch accuracy: 0.886 Validation accuracy: 0.8762
5 Batch accuracy: 0.87 Validation accuracy: 0.884
6 Batch accuracy: 0.91 Validation accuracy: 0.8922
7 Batch accuracy: 0.894 Validation accuracy: 0.8964
8 Batch accuracy: 0.922 Validation accuracy: 0.901
9 Batch accuracy: 0.892 Validation accuracy: 0.9054
10 Batch accuracy: 0.9 Validation accuracy: 0.9088
11 Batch accuracy: 0.918 Validation accuracy: 0.9116
12 Batch accuracy: 0.9 Validation accuracy: 0.9132
13 Batch accuracy: 0.914 Validation accuracy: 0.9172
14 Batch accuracy: 0.916 Validation accuracy: 0.919
15 Batch accuracy: 0.916 Validation accuracy: 0.9196
16 Batch accuracy: 0.908 Validation accuracy: 0.9224
17 Batch accuracy: 0.912 Validation accuracy: 0.924
18 Batch accuracy: 0.898 Validation accuracy: 0.9258
19 Batch accur

In [0]:
with tf.Session() as sess:
    saver.restore(sess, final_model_path)
    train_Accuracy_val = accuracy.eval(feed_dict={X: X_train, y: y_train})
    test_Accuracy_val = accuracy.eval(feed_dict={X: X_test, y: y_test})
    
print ("Training Accuracy=",train_Accuracy_val)
print ("Testing Accuracy=",test_Accuracy_val)

INFO:tensorflow:Restoring parameters from ./model2
Training Accuracy= 0.9689091
Testing Accuracy= 0.9644


In [0]:
show_graph(tf.get_default_graph().as_graph_def())

In [0]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [0]:
with tf.name_scope("dnn3"):
    hidden1 = tf.layers.dense(X, n_hidden2_1, name="hidden1",
                              activation=tf.nn.sigmoid, use_bias=True)
    hidden2 = tf.layers.dense(hidden1, n_hidden2_2, name="hidden2",
                              activation=tf.nn.sigmoid, use_bias=True)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

In [0]:
with tf.name_scope("loss3"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss") 

In [0]:
learning_rate = 0.01

with tf.name_scope("train3"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval3"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [0]:
n_epochs = 120
batch_size = 500
final_model_path = "./model3"

start_tm=datetime.datetime.now()
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        saver.save(sess, final_model_path)
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)
end_tm=datetime.datetime.now()
print ("Time taken to fit is ",end_tm - start_tm)

0 Batch accuracy: 0.122 Validation accuracy: 0.1126
1 Batch accuracy: 0.148 Validation accuracy: 0.1126
2 Batch accuracy: 0.122 Validation accuracy: 0.1126
3 Batch accuracy: 0.106 Validation accuracy: 0.1126
4 Batch accuracy: 0.114 Validation accuracy: 0.1126
5 Batch accuracy: 0.132 Validation accuracy: 0.1126
6 Batch accuracy: 0.114 Validation accuracy: 0.1126
7 Batch accuracy: 0.108 Validation accuracy: 0.1126
8 Batch accuracy: 0.126 Validation accuracy: 0.1126
9 Batch accuracy: 0.104 Validation accuracy: 0.1126
10 Batch accuracy: 0.106 Validation accuracy: 0.1126
11 Batch accuracy: 0.12 Validation accuracy: 0.1126
12 Batch accuracy: 0.124 Validation accuracy: 0.1126
13 Batch accuracy: 0.088 Validation accuracy: 0.1126
14 Batch accuracy: 0.124 Validation accuracy: 0.1126
15 Batch accuracy: 0.128 Validation accuracy: 0.1126
16 Batch accuracy: 0.102 Validation accuracy: 0.1126
17 Batch accuracy: 0.104 Validation accuracy: 0.1126
18 Batch accuracy: 0.116 Validation accuracy: 0.1126
19 B

In [0]:
with tf.Session() as sess:
    saver.restore(sess, final_model_path)
    train_Accuracy_val = accuracy.eval(feed_dict={X: X_train, y: y_train})
    test_Accuracy_val = accuracy.eval(feed_dict={X: X_test, y: y_test})
    
print ("Training Accuracy=",train_Accuracy_val)
print ("Testing Accuracy=",test_Accuracy_val)

INFO:tensorflow:Restoring parameters from ./model3
Training Accuracy= 0.11234546
Testing Accuracy= 0.1135


In [0]:
show_graph(tf.get_default_graph().as_graph_def())

In [0]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [0]:
with tf.name_scope("dnn4"):
    hidden1 = tf.layers.dense(X, n_hidden2_1, name="hidden1",
                              activation=tf.nn.relu, use_bias=True)
    hidden2 = tf.layers.dense(hidden1, n_hidden2_2, name="hidden2",
                              activation=tf.nn.relu, use_bias=True)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

In [0]:
with tf.name_scope("loss4"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss") 

In [0]:
learning_rate = 0.01

with tf.name_scope("train4"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval4"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [0]:
n_epochs = 120
batch_size = 500
final_model_path = "./model4"

start_tm=datetime.datetime.now()
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        saver.save(sess, final_model_path)
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        #acc_valid = accuracy.eval(feed_dict={X: X_train, y: y_train})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)
end_tm=datetime.datetime.now()
print ("Time taken to fit is ",end_tm - start_tm)

0 Batch accuracy: 0.656 Validation accuracy: 0.6752
1 Batch accuracy: 0.804 Validation accuracy: 0.7998
2 Batch accuracy: 0.83 Validation accuracy: 0.8404
3 Batch accuracy: 0.868 Validation accuracy: 0.8618
4 Batch accuracy: 0.874 Validation accuracy: 0.8724
5 Batch accuracy: 0.864 Validation accuracy: 0.8834
6 Batch accuracy: 0.898 Validation accuracy: 0.8908
7 Batch accuracy: 0.9 Validation accuracy: 0.897
8 Batch accuracy: 0.918 Validation accuracy: 0.9018
9 Batch accuracy: 0.88 Validation accuracy: 0.9048
10 Batch accuracy: 0.894 Validation accuracy: 0.9086
11 Batch accuracy: 0.904 Validation accuracy: 0.9106
12 Batch accuracy: 0.914 Validation accuracy: 0.9116
13 Batch accuracy: 0.902 Validation accuracy: 0.9154
14 Batch accuracy: 0.904 Validation accuracy: 0.9172
15 Batch accuracy: 0.906 Validation accuracy: 0.9184
16 Batch accuracy: 0.918 Validation accuracy: 0.9202
17 Batch accuracy: 0.904 Validation accuracy: 0.9216
18 Batch accuracy: 0.89 Validation accuracy: 0.923
19 Batch a

In [0]:
with tf.Session() as sess:
    saver.restore(sess, final_model_path)
    train_Accuracy_val = accuracy.eval(feed_dict={X: X_train, y: y_train})
    test_Accuracy_val = accuracy.eval(feed_dict={X: X_test, y: y_test})
    
print ("Training Accuracy=",train_Accuracy_val)
print ("Testing Accuracy=",test_Accuracy_val)

INFO:tensorflow:Restoring parameters from ./model4
Training Accuracy= 0.96450907
Testing Accuracy= 0.9603


In [0]:
show_graph(tf.get_default_graph().as_graph_def())